In [1]:
from dolfin import *
import numpy as np

In [2]:
# 参数准备
nelx, nely = 10, 10
# 二维正交各向异性
E1, E2, mu12, G12 = 130000, 7700, 0.33, 4800
# 转动角度
Theta = 0
Theta_init = 5
# 载荷
forcing = Constant((0, -500))

In [3]:
# 弹性张量
C2D_Iso = np.array([
    [E1/(1-mu12**2),        mu12*E2/(1-mu12**2),         0  ],
    [mu12*E2/(1-mu12**2),   E2/(1-mu12**2),              0  ],
    [0,                     0,                           G12]
])



# 转轴公式
def T2D_inv(theta):
    theta = theta * np.pi / 180.0
    c = cos(theta)
    s = sin(theta)
    Trans = np.array([
        [c**2,  s**2,   -2*s*c],
        [s**2,  c**2,   2*s*c],
        [s*c,   -s*c,   c**2 - s**2]
    ])
    # Trans = np.array([
    #     [np.cos(theta)**2, np.sin(theta)**2, -2*np.sin(theta)*np.cos(theta)],
    #     [np.sin(theta)**2, np.cos(theta)**2, 2*np.sin(theta)*np.cos(theta)],
    #     [np.sin(theta)*np.cos(theta), -np.sin(theta)*np.cos(theta), np.cos(theta)**2-np.sin(theta)**2]
    # ])
    return Trans

In [12]:
# 模型准备
mesh = UnitSquareMesh(nelx, nely)
V = VectorFunctionSpace(mesh, "CG", 1)
T = FunctionSpace(mesh, "DG", 0)
print(T.dim())
u_sol = Function(V)
Theta_sol = Function(T)
# v = Function(T)
# v.vector()[:] = 1
u_trial = TrialFunction(V)
v_test = TestFunction(V)


200


In [5]:
# 物理方程
def epsilon(u):
    engineering_strain = 0.5 * (nabla_grad(u) + nabla_grad(u).T)
    return engineering_strain

# 本构方程
def sigma_tensor(u, Theta_Ev):
    # 计算应变张量
    epsilon_ij = epsilon(u)
    ep = as_vector([epsilon_ij[0,0], epsilon_ij[1,1], epsilon_ij[0,1]])
    # 使用弹性系数计算应力张量
    Q_bar = np.dot(np.dot(T2D_inv(Theta_Ev), C2D_Iso), T2D_inv(Theta_Ev).T)
    sigma_ij = np.dot(Q_bar, ep)

    return as_tensor([[sigma_ij[0], sigma_ij[2]],
                      [sigma_ij[2], sigma_ij[1]]])

# 弹性能密度
def psi(u, Theta):
    return 0.5 * inner(sigma_tensor(u, Theta), epsilon(u))

In [6]:
# 边界定义
# 狄利克雷边界
def clamped_boundary(x, on_boundary):
    return on_boundary and x[0] < DOLFIN_EPS

bc = DirichletBC(V, Constant((0.0, 0.0)), clamped_boundary)

# 自然边界
class RightEnd(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 1) < DOLFIN_EPS and (abs(x[1] - 0.5) < 0.5)
right_end_boundary = RightEnd()

class TopEnd(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0] - 0.5) < 0.1 and (abs(x[1] - 1) < DOLFIN_EPS)
top_end_boundary = TopEnd()

boundary_mark = MeshFunction("size_t", mesh, mesh.topology().dim()-1)
boundary_mark.set_all(0)
right_end_boundary.mark(boundary_mark, 1)
top_end_boundary.mark(boundary_mark, 2)

In [7]:
lhs = inner(sigma_tensor(u_trial, Theta_sol), nabla_grad(v_test)) * dx
rhs = dot(forcing, v_test) * ds(subdomain_data=boundary_mark, domain=mesh, subdomain_id=1)

In [8]:
# 有限元求解
def FEA(V:VectorFunctionSpace, lhs, rhs, u:Function, bc:DirichletBC) -> Function: 
    problem = LinearVariationalProblem(lhs, rhs, u, bc)
    solver = LinearVariationalSolver(problem)
    return solver.solve()


In [11]:
# 敏度计算
def compute_sensetivity(u, Theta):
    W = 0.5 * inner(sigma_tensor(u, Theta), epsilon(u)) * dx
    J = assemble(W)
    dJ_dTheta = derivative(Constant(J), Theta)

    # 将敏度信息组装成向量
    dJ_dTheta_vec = assemble(dJ_dTheta)
    return dJ_dTheta_vec

In [10]:
if __name__ == "__main__":
    # Adam 优化参数
    learning_rate = 0.01
    beta1 = 0.9
    beta2 = 0.999
    epsilon_adam = 1e-8
    max_iter = 10000
    tol = 1e-6

    Theta_sol.assign(interpolate(Constant(Theta_init), T))
    m = Function(T)
    v = Function(T)
    m.vector()[:] = 0.0
    v.vector()[:] = 0.0

    for i in range(max_iter):
        FEA(V, lhs, rhs, u_sol, bc)
        W = FunctionSpace(mesh, "DG", 0)
        psi_re = project(psi(u_sol, Theta_sol), W)
        total_energy = assemble(psi_re * dx)

        print(f"Iteration {i}, Total Energy: {total_energy}")

        # 计算目标函数 J
        J = psi(u_sol, Theta_sol) * dx
        
        # 计算目标函数对 Theta 的导数
        dJ_dTheta =  (J, Theta_sol)
        
        # # 将导数投影到函数空间 T 中
        # # dpsi_dTheta = project(dJ_dTheta, T)
        # dJ_dTheta_vec = assemble(dJ_dTheta)
        # # print(type(dJ_dTheta_vec))
        sensitivity = compute_sensetivity(u_sol, Theta)
        
        # 将 Vector 对象转换为 numpy 数组
        m_np = m.vector().get_local()
        v_np = v.vector().get_local()
        dJ_dTheta_np = dJ_dTheta_vec.get_local()

        # Adam 优化步骤
        m.vector()[:] = beta1 * m.vector()[:] + (1 - beta1) * dJ_dTheta_np
        v.vector()[:] = beta2 * v.vector()[:] + (1 - beta2) * (dJ_dTheta_np ** 2)
        m_hat = m.vector()[:] / (1 - beta1 ** (i + 1))
        v_hat = v.vector()[:] / (1 - beta2 ** (i + 1))
        
        update = learning_rate * m_hat / (np.sqrt(v_hat) + epsilon_adam)
        
        # 创建一个用于更新的 GenericVector
        update_vec = Function(T)
        update_vec.vector()[:] = update
        
        # 使用 axpy 方法更新 Theta_sol
        Theta_sol.vector().axpy(-1.0, update_vec.vector())
        
        # 检查收敛
        if np.linalg.norm(update) < tol:
            break

    print(Theta_sol.vector()[:])

Solving linear variational problem.
Iteration 0, Total Energy: 37.69951215722479


AttributeError: 'float' object has no attribute 'arguments'

In [ ]:
# 绘制收敛曲线
import matplotlib.pyplot as plt
plt.figure()
plt.plot(total_energy_list)
plt.xlabel("Iteration")
plt.ylabel("Total Energy")
plt.title("Convergence of Total Energy")
plt.show()

# 绘制结果
plot(Theta_sol, title="Optimized Fiber Angle")
plot(u_sol, title="Displacement Field")
plot(psi_re, title="Strain Energy Density")
plt.show()